### Credit Assignment Problem
* Actions naturally will have delayed effects. No way of being certain how much an action contributed to a result
I.e Winning a point in a game, was it just the last action that contributed to the result or an earlier action(s)?
#### Policy Gradient Algorithm 
This algorithm seems to tackle the credit assigment problem b
* Playing multiple games
* Then making the actions in good games slightly more likely
* Making actions in bad games slightly less likely

In [1]:
import numpy as np
import os
import sys

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures and animations
%matplotlib nbagg
import matplotlib
import matplotlib.animation as animation
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

import tensorflow as tf
import gym

n_inputs = 4
n_hidden = 4
n_outputs = 1
learning_rate = 0.01
reset_graph()

initializer = tf.contrib.layers.variance_scaling_initializer()
X = tf.placeholder(tf.float32, shape=[None, n_inputs])

hidden = tf.layers.dense(X, n_hidden, activation=tf.nn.elu, kernel_initializer=initializer)
logits = tf.layers.dense(hidden, n_outputs)
outputs =tf.nn.sigmoid(logits)
p_left_and_right = tf.concat(axis=1, values=[outputs, 1-outputs])
action = tf.multinomial(tf.log(p_left_and_right), num_samples=1)

# target probability
# We are acting as though the chosen action is the best possible action. 
# Therefore, the target probability == 1 if chosen action is 0 (left) 
#            and the target probability == 0.0 if chosen action is 1 (right)
y = 1. - tf.to_float(action)
cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=logits)
optimizer = tf.train.AdamOptimizer(learning_rate)
# We want to tweak the graidents before applyinng them. We compute into a seperate variable first
# compute_gradients returns gradient vector/variable pairs - a pair per training variable.
grads_and_vars = optimizer.compute_gradients(cross_entropy)

# put all in gradients in a list
gradients = [grad for grad, variable in grads_and_vars]

# During execution the algorithm will run the policy and at each step evaluate the graident tensors and store
# their values
# Afer a number of episodes it will
# * Tweak these gradients - by multipying them by the action scores and normalise them. Then compute the mean
#    of the tweaked gradients. 
# * Feed the resulting gradients back to the optimizer so that it can perfom an optimization step

gradient_placeholders = []
grads_and_vars_feed = []
for grad, variable in grads_and_vars:
    gradient_placeholder = tf.placeholder(tf.float32, shape=grad.get_shape())
    gradient_placeholders.append(gradient_placeholder)
    grads_and_vars_feed.append((gradient_placeholder, variable))

training_op = optimizer.apply_gradients(grads_and_vars_feed)


init = tf.global_variables_initializer()
saver = tf.train.Saver()

### Execution


In [2]:
def discount_rewards(rewards, discount_rate):
    discounted_rewards = np.empty(len(rewards))
    cumulative_rewards = 0
    for step in reversed(range(len(rewards))):
        cumulative_rewards = rewards[step] + cumulative_rewards * discount_rate
        discounted_rewards[step] = cumulative_rewards
    return discounted_rewards

def discount_and_normalize_rewards(all_rewards, discount_rate):
    all_discounted_rewards = [discount_rewards(rewards, discount_rate) for rewards in all_rewards]
    flat_rewards = np.concatenate(all_discounted_rewards)
    reward_mean = flat_rewards.mean()
    reward_std = flat_rewards.std()
    return [(discounted_rewards - reward_mean)/reward_std for discounted_rewards in all_discounted_rewards]

In [3]:
discount_rewards([10, 0, -50], discount_rate=0.8)

array([-22., -40., -50.])

In [4]:
discount_and_normalize_rewards([[10, 0, -50], [10, 20]], discount_rate=0.8)

[array([-0.28435071, -0.86597718, -1.18910299]),
 array([ 1.26665318,  1.0727777 ])]

In [7]:
env = gym.make('CartPole-v0')

n_games_per_update = 10
n_max_steps = 1000
n_iterations = 250
save_iterations = 10
discount_rate = 0.95

with tf.Session() as sess:
    init.run()
    for iteration in range(n_iterations):
        print('\rIteration: {}'.format(iteration), end='')
        all_rewards = []
        all_gradients = []
        for step in range(n_games_per_update):
            current_rewards = []
            current_gradients = []
            obs = env.reset()
            for step in range(n_max_steps):
                action_val, gradients_val = sess.run([action, gradients], feed_dict={X:obs.reshape(1, n_inputs)})
                obs, reward, done, info = env.step(action_val[0][0])
                current_rewards.append(reward)
                current_gradients.append(gradients_val)
                if done:
                    break
            all_rewards.append(current_rewards)
            all_gradients.append(current_gradients)
        all_rewards = discount_and_normalize_rewards(all_rewards, discount_rate=discount_rate)
        feed_dict = {}
        for var_index, gradient_placehoder in enumerate(gradient_placeholders):
            mean_gradients = np.mean([reward * all_gradients[game_index][step][var_index]
                                      for game_index, rewards in enumerate(all_rewards)
                                            for step, reward in enumerate(rewards)], axis=0)
            feed_dict[gradient_placeholder] = mean_gradients

        # for var_index, gradient_placeholder in enumerate(gradient_placeholders):
        #     mean_gradients = np.mean([reward * all_gradients[game_index][step][var_index]
        #                               for game_index, rewards in enumerate(all_rewards)
        #                                   for step, reward in enumerate(rewards)], axis=0)
        #     feed_dict[gradient_placeholder] = mean_gradients
        # 
        sess.run(training_op, feed_dict=feed_dict)
        if iteration % save_iterations == 0:
            saver.save(sess,'./my_policy_net_pg.ckpt')

[2017-06-10 16:48:08,394] Making new env: CartPole-v0


Iteration: 0

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_1' with dtype float and shape [4,4]
	 [[Node: Placeholder_1 = Placeholder[dtype=DT_FLOAT, shape=[4,4], _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op 'Placeholder_1', defined at:
  File "/home/dayo/.conda/envs/py3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/dayo/.conda/envs/py3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/dayo/.conda/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/dayo/.conda/envs/py3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/dayo/.conda/envs/py3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home/dayo/.conda/envs/py3/lib/python3.6/site-packages/zmq/e

In [5]:
# env = gym.make("CartPole-v0")
# 
# n_games_per_update = 10
# n_max_steps = 1000
# n_iterations = 250
# save_iterations = 10
# discount_rate = 0.95
# 
# with tf.Session() as sess:
#     init.run()
#     for iteration in range(n_iterations):
#         print("\rIteration: {}".format(iteration), end="")
#         all_rewards = []
#         all_gradients = []
#         for game in range(n_games_per_update):
#             current_rewards = []
#             current_gradients = []
#             obs = env.reset()
#             for step in range(n_max_steps):
#                 action_val, gradients_val = sess.run([action, gradients], feed_dict={X: obs.reshape(1, n_inputs)})
#                 obs, reward, done, info = env.step(action_val[0][0])
#                 current_rewards.append(reward)
#                 current_gradients.append(gradients_val)
#                 if done:
#                     break
#             all_rewards.append(current_rewards)
#             all_gradients.append(current_gradients)
# 
#         all_rewards = discount_and_normalize_rewards(all_rewards, discount_rate=discount_rate)
#         feed_dict = {}
#         for var_index, gradient_placeholder in enumerate(gradient_placeholders):
#             mean_gradients = np.mean([reward * all_gradients[game_index][step][var_index]
#                                       for game_index, rewards in enumerate(all_rewards)
#                                           for step, reward in enumerate(rewards)], axis=0)
#             feed_dict[gradient_placeholder] = mean_gradients
#         sess.run(training_op, feed_dict=feed_dict)
#         if iteration % save_iterations == 0:
#             saver.save(sess, "./my_policy_net_pg.ckpt")

[2017-06-10 16:38:05,038] Making new env: CartPole-v0


Iteration: 0

NameError: name 'saver' is not defined